In [11]:
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=932f7a3f3a98034b7be8d169e010132498dbaf71884a30c1d18a1bce147bb00d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9bee

In [12]:
import pandas as pd

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)


In [2]:
QPATH = "Quantlet/EDA_and_script_similarity"
PATH = f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}'

In [3]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(PATH)

sys.path.append('../src')

In [6]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch

import nltk
nltk.download('punkt')

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
train = pd.read_csv('../../data/preprocessed/Quantlet/train_df.csv')
val   = pd.read_csv('../../data/preprocessed/Quantlet/val_df.csv')
test  = pd.read_csv('../../data/preprocessed/Quantlet/test_df.csv')

In [10]:
train['code_n_words'] = train.code_script.apply(lambda x: len(x.split()))
val['code_n_words'] = val.code_script.apply(lambda x: len(x.split()))
test['code_n_words'] = test.code_script.apply(lambda x: len(x.split()))

In [13]:
model_name = "SEBIS/code_trans_t5_base_source_code_summarization_python_multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)


#source_tokenized = tokenizer(
#        source,
#        padding="max_length",
#        truncation=True,
#        max_length=max_source_length
#    )

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [15]:
train.code_n_words.describe()

count    5537.000000
mean      313.746794
std       399.053636
min         1.000000
25%        91.000000
50%       172.000000
75%       376.000000
max      2500.000000
Name: code_n_words, dtype: float64

In [18]:
train.code_n_words.head()

0    356
1    591
2    647
3     91
4    647
Name: code_n_words, dtype: int64

In [23]:
len(tokenizer(train['code_script'].iloc[1])['input_ids'])

2524

In [26]:
(train['code_script'].iloc[1])

'######################################### ## visualization of simulation results ## par(bg="transparent") #------------------------------------------------------------------------------- # stacked bar plot of spreads and fees plot_spreads_fees <- function(){ par(bg="transparent") #aggregate spreads and fees #take 95% quantile fees <- aggregate(step3$costs, by=list(step3$date), function(x)quantile(x,probs = 0.95)) spreads <- aggregate(step3$spread.10, by=list(step3$date), function(x)quantile(x,probs = 0.5)) fees$type <- "trading_fee" names(fees) <- c("date", "quantile", "type") spreads$type <- "spread" names(spreads) <- c("date", "quantile", "type") #merge sf.df <- rbind(fees, spreads) sf.df$type <- factor(sf.df$type) sf.df$date <- as.Date(sf.df$date, format="%Y-%m-%d") #plot ggplot(sf.df, aes(fill=type, y=quantile, x=date)) + geom_bar(position="stack", stat="identity")+ scale_fill_viridis(discrete = T, begin = 0.5) + ylab("")+ #scale_x_date(limits = c(as.Date("2020-07-01"), theme( pan

In [ ]:
tokenizer.tokenize(train['code_script'].iloc[1])

In [34]:
source_tokenized = tokenizer.tokenize(
        train['Description'].iloc[1],
        padding="max_length",
        truncation=True,
        max_length=512
)